Value-added graph - graph can be filtered and grouped by Day, Month and Year

In [ ]:
%pip install jupyter-dash
%pip install dash

In [ ]:
import requests
import pandas as pd
import plotly.express as px

In [ ]:
# hardcoded singapore into the url to only see singapore's data
url = "https://api.covid19api.com/total/country/singapore/status/confirmed"
response = requests.get(url)

In [ ]:
from datetime import date,timedelta,datetime
def check_last_month_day(day):
    next_day = day + timedelta(days=1)
    return next_day.day == 1
def check_last_year_day(day):
    next_day = day + timedelta(days=1)
    return next_day.day == 1 and next_day.month == 1

In [ ]:
df = pd.json_normalize(response.json())
df_cleaned = df[['Date', 'Cases']]
df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'])
df_cleaned['Date_flat'] = df_cleaned['Date'].dt.date

In [ ]:
from jupyter_dash import JupyterDash
from dash import html,dcc
from dash.dependencies import Input, Output

# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Covid Dashboard"),
    html.Label([
        "View Covid Cases By",
        dcc.Dropdown(
            id='daterange-dropdown', clearable=False,
            value='Day', options=[
                {'label':'Day','value':'Day'},
                {'label':'Month','value':'Month'},
                {'label':'Year','value':'Year'}
            ])
        ]),
    dcc.DatePickerRange(
        id='my-date-picker-range',
        min_date_allowed=date(2020, 1,22),
        max_date_allowed=date(2022, 10, 7),
        initial_visible_month=date(2020, 1, 22),
        end_date=date(2022, 10, 7)
    ),
    html.Div(id='output-container-date-picker-range'),
    dcc.Graph(id='graph')
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    Input("daterange-dropdown", "value"),
    Input('my-date-picker-range', 'start_date'),
    Input('my-date-picker-range', 'end_date')
)
def update_figure(year_choice, start_date, end_date):
    if start_date is None:
        start_date = date(2020,1,22)
    else:
        start_date = date.fromisoformat(start_date)
    if end_date is None:
        end_date = date(2022,10,7)
    else:
        end_date = date.fromisoformat(end_date)
    df1 = df_cleaned[(df_cleaned['Date_flat'] >= start_date) & (df_cleaned['Date_flat'] <= end_date)]
    df_last_day_of_month = df1[df1['Date'].apply(check_last_month_day)][['Date','Cases']]
    df_last_day_of_month['DateMonth'] = df1['Date'].dt.to_period('M').astype(str)
    frames = [df1[df1['Date'].apply(check_last_year_day)][['Date','Cases']],df1[['Date','Cases']].tail(1)]
    df_last_day_of_year = pd.concat(frames)
    df_last_day_of_year['Year'] = df1['Date'].dt.to_period('Y').astype(str)
    if year_choice == 'Day':
        return px.line(df1, x='Date', y='Cases',title='Number of COVID-19 cases in Singapore over time')
    elif year_choice == 'Month':
        return px.line(df_last_day_of_month, x='DateMonth', y='Cases', title='Number of COVID-19 cases in Singapore over time by months')
    else:
        return px.line(df_last_day_of_year, x='Year', y='Cases', title='Number of COVID-19 cases in Singapore over time in Years')
    
# Run app and display result inline in the notebook
app.run_server(mode='inline')
